In [ ]:
# pip install boto3
import boto3, time # see https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lightsail.html 

# You input is necessary

* Choose region_name
* paste aws id, aws key and domainName
* Change domain registration according to the name servers listed in the end

In [ ]:
region_name='us-east-1' 
# us-east-2 @ Ohio
# us-east-1 @ N. Virginia
# us-west-2 @ Oregon
# ap-south-1 @ Mumbai
# ap-northeast-2 @ Seoul
# ap-southeast-1 @ Singapore
# ap-southeast-2 @ Sydney
# ap-northeast-1 @ Tokyo
# ca-central-1 @ Central
# eu-central-1 @ Frankfurt
# eu-west-1 @ Ireland
# eu-west-2 @ London
# eu-west-3 @ Paris

In [ ]:
##### Learn how to get these info
##### https://docs.aws.amazon.com/powershell/latest/userguide/pstools-appendix-sign-up.html
aws_access_key_id='AKIAMMLKFNWRZDLRJ' 
aws_secret_access_key='97i8FNOUEIBw3b1/VEUb9w31/014rQ3egBJN' 

In [ ]:
domainName = 'domain.name'

# Operations

### Setup Client for the region of N. Virginia

In [ ]:
client = boto3.client('lightsail', region_name=region_name, use_ssl=True, 
                      aws_access_key_id=aws_access_key_id, 
                      aws_secret_access_key=aws_secret_access_key)

### Create SSH Key pair and print private key

In [ ]:
response = client.create_key_pair(keyPairName='new_ssh_key')
print(response['privateKeyBase64'])

### Create Instance in N. Virginia

In [ ]:
response = client.create_instances(
    instanceNames=['created_from_python',],
    availabilityZone=region_name+'a',
    blueprintId='ubuntu_18_04',
    bundleId='nano_2_0', #  micro_2_0
    userData='echo "success">/home/ubuntu/test.log', # Initialization shellscript
    keyPairName='new_ssh_key'
) 

while 1:
    response = client.get_instance_state(instanceName='created_from_python')
    if response['state']['name'] == 'running': 
        print('New Instance is ready~')
        break
    time.sleep(5)

### Allocate static ip

In [ ]:
response = client.allocate_static_ip(staticIpName='static_ip_from_python')

##### Get Static ip

In [ ]:
response = client.get_static_ip(
    staticIpName = 'static_ip_from_python'
)
static_ip = response['staticIp']['ipAddress']
print(static_ip)

### Attach static ip to instance

In [ ]:
response = client.attach_static_ip(
    staticIpName='static_ip_from_python',
    instanceName='created_from_python'
)

### Open a port on Firewall 

In [ ]:
response = client.open_instance_public_ports(
    portInfo={
        'fromPort': 443,
        'toPort': 443,
        'protocol': 'tcp',
        'cidrs': [ "0.0.0.0/0", ],
    },
    instanceName='created_from_python'
)

### Create a DNS_domain/DNS_zone then get the nameservers

In [ ]:
response = client.create_domain(domainName=domainName)
_ = [print(i['target']) for i in client.get_domain( domainName=domainName)['domain']['domainEntries'] if i['type'] == 'NS']

### Go to domain registrar and change nameserver

### Create 'A' DNS record then access the server with ssh private key (username: ubuntu)

In [ ]:
response = client.create_domain_entry(
    domainName=domainName,
    domainEntry={ 
        'name': domainName,
        'target': static_ip, 
        'type': 'A', 
    }
)

### Delete Instance

In [ ]:
response = client.delete_instance(
    instanceName='created_from_python', 
)

### Delete static ip

In [ ]:
response = client.release_static_ip(
    staticIpName='static_ip_from_python'
)

### Start instance

In [ ]:
response = client.start_instance(
    instanceName='created_from_python'
)

while 1:
    response = client.get_instance_state(instanceName='created_from_python')
    if response['state']['name'] == 'running': 
        print('New Instance is ready~')
        break
    time.sleep(5)

### Stop instance

In [ ]:
response = client.stop_instance(
    instanceName='created_from_python'
)

while 1:
    response = client.get_instance_state(instanceName='created_from_python')
    if response['state']['name'] == 'stopped': 
        print('Instance stopped~')
        break
    time.sleep(5)